<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/TrigramLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import random
import math

In [5]:
import requests

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
url = 'https://raw.githubusercontent.com/karpathy/makemore/master/names.txt'
url = 'https://raw.githubusercontent.com/Trickshotblaster/nn-practices/main/TrigramLM.ipynb'
response = requests.get(url)

with open('file.txt', 'wb') as f:
  f.write(response.content)

In [6]:
with open('file.txt', 'r') as data:
  dataset = data.readlines()

with open('file.txt', 'r') as data:
  raw_data = data.read()


In [7]:
raw_data[:10]

'{\n  "nbfor'

In [8]:
dataset = raw_data.split("\n")

In [9]:
dataset[:10]

['{',
 '  "nbformat": 4,',
 '  "nbformat_minor": 0,',
 '  "metadata": {',
 '    "colab": {',
 '      "provenance": [],',
 '      "include_colab_link": true',
 '    },',
 '    "kernelspec": {',
 '      "name": "python3",']

In [10]:
train_set = dataset[:math.floor(len(dataset) * 0.8)]
dev_set = dataset[math.floor(len(dataset) * 0.8):math.floor(len(dataset) * 0.9)]
test_set = dataset[math.floor(len(dataset) * 0.9):len(dataset) - 1]

In [11]:
vocab = set(raw_data)

In [12]:
vocab_len = len(vocab)

In [13]:
vocab_len

86

In [14]:
stoi = {char:i for i, char in enumerate(vocab)}

In [15]:
itos = {i:char for i, char in enumerate(vocab)}

In [16]:
import torch

In [17]:
import torch.nn.functional as functional

In [18]:
# make dataset
xs, ys = [], []
for data in train_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

34118

In [19]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([172, 86])

In [20]:
xenc = functional.one_hot(xs[0], num_classes=vocab_len).float().flatten()
xenc @ ws

tensor([ 1.0794,  1.8022,  1.9395, -0.5692,  0.0620, -3.7957, -1.0224, -1.3213,
        -0.5041, -0.2540, -1.3792,  1.5899,  1.5425,  0.4421,  1.6314, -0.5042,
         0.3970, -0.0475, -0.2891, -0.0353, -1.7860,  2.1028,  0.2761, -1.5458,
        -0.1144,  0.9313,  0.5265, -0.2161,  0.1975,  0.1169, -0.7408, -2.0907,
        -0.2515, -0.7268,  1.3164, -0.8426,  0.2475, -0.2823,  1.1661,  0.9416,
         1.4624, -0.5405, -1.9547,  0.0190, -0.4224, -1.7860, -0.9386, -0.7907,
         1.1146,  1.5420,  0.2392,  2.6738,  0.5329,  1.1459, -0.3995, -0.2969,
        -0.9814,  0.9496,  1.5875, -1.1933,  2.0189,  0.7296,  0.8806,  0.7074,
         0.7530, -2.6069, -1.8081,  0.1439, -2.0136, -1.4745, -0.3157,  2.2345,
         1.3309, -0.6457,  1.4138, -0.2124,  0.3019, -0.5609,  0.3886, -2.0784,
         1.1653, -0.4307,  1.0796, -1.3916,  0.7843,  0.1512],
       grad_fn=<SqueezeBackward4>)

In [21]:
(xenc @ ws).shape

torch.Size([86])

In [22]:
num_epochs = 100
lr = 20

In [23]:
torch.randn((3, 1))

tensor([[1.4632],
        [0.3529],
        [1.5197]])

In [24]:
for epoch in range(num_epochs):

    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * 0.01)
    print(loss.item())

    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr

4.968534469604492
4.1083598136901855
3.842843532562256
3.6717939376831055
3.5867834091186523
3.510815382003784
3.4395248889923096
3.3759772777557373
3.314143419265747
3.2641849517822266
3.208268404006958
3.170440196990967
3.1164026260375977
3.085538387298584
3.0354416370391846
3.009063959121704
2.963214635848999
2.9405465126037598
2.8979837894439697
2.878469705581665
2.8386282920837402
2.8218278884887695
2.7843074798583984
2.769847869873047
2.7343356609344482
2.721897602081299
2.6881449222564697
2.6774637699127197
2.6452724933624268
2.636132001876831
2.60534405708313
2.5975711345672607
2.5680596828460693
2.5615155696868896
2.533174753189087
2.527745246887207
2.5004818439483643
2.4960715770721436
2.469801187515259
2.4663264751434326
2.4409706592559814
2.438354730606079
2.41383957862854
2.4120116233825684
2.388268232345581
2.3871634006500244
2.3641257286071777
2.363682270050049
2.3412930965423584
2.34145450592041
2.3196609020233154
2.320375680923462
2.2991323471069336
2.300353527069092
2

In [25]:
xs[:10]

tensor([[53, 53],
        [53, 13],
        [13, 43],
        [43, 26],
        [26,  9],
        [ 9,  2],
        [ 2, 75],
        [75, 73],
        [73, 74],
        [74, 38]])

In [26]:
def forward(xs):
  xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
  logits = xenc @ ws
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  outchar = itos[torch.multinomial(probs, num_samples=1).item()]
  return outchar

def prompt(ins):
  out = ins
  for char in range(500):
    last_two = ins[-2:]
    xs = torch.tensor([[stoi[last_two[0]], stoi[last_two[1]]]])
    outs = forward(xs)
    out += outs
  return out

prompt("Rom")

'Romrkrw"eae[ieaee__@,vee_\newteer_rdr*seeLSe:iaeUseevs_BwaaeUreraear_ea}IaiOaeeaaUiaea_aaerz_eeegeai_asa_Uea[.aee[a[eie.Leaerre@<eeQa_e_eTrkImee_sreks)a_66eUr_[eu_er<a_aer_rCeE___1a_ere__erea_eaeeaiaOeee)Is[iieese[ee_v-aa_iaaeea_aa/[e/Ueeke_aa_eGai<iea_v8eaeeae__ex_Ce[__eVO/[[eaeeeQu_eeesieBad7xac/i[as__retDeeUe?re_a[aeevrVreeC[ear_e0ieUacrBXro_eaLe[eeQeeecGvC[DU_"iaueem_aeC_re/ea eeeee1e[6Sia aeaa[9e5evaC__L<ee2[aaEeeeiiWev[=Oe[_[_B_eweeaaa_a_ae_[rsieeei_ee0eeDaD"asne[[sXe<W_are_[araevhveLLXasDeaBr'

In [27]:
# make test set
xs, ys = [], []
for data in test_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

3936

In [28]:
xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
logits = xenc @ ws
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * 0.01)
print(loss.item())

2.4101316928863525


In [29]:
# make dev set
xs, ys = [], []
for data in dev_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

3850

In [30]:
xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
logits = xenc @ ws
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * 0.01)
print(loss.item())

2.1572365760803223


Looks like it might be overfitting? The dev set results seem fine, but the test set loss it much higher than it should be.

In [33]:
# make dev set
dxs, dys = [], []
for data in dev_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    dxs.append([stoi[ch1], stoi[ch2]])
    dys.append(stoi[ch3])

dxs = torch.tensor(xs)
dys = torch.tensor(ys)
dnum = xs.nelement()
dnum

<ipython-input-33-af72f93a7454>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dxs = torch.tensor(xs)
<ipython-input-33-af72f93a7454>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dys = torch.tensor(ys)


3850

In [31]:
num_epochs = 100
lr = 20
regularization = 0.01

In [38]:
loss = 2.1

for epoch in range(num_epochs):
    dxenc = functional.one_hot(dxs, num_classes=vocab_len).float().flatten(start_dim=1)
    dlogits = dxenc @ ws
    dcounts = dlogits.exp()
    dprobs = dcounts / dcounts.sum(1, keepdim=True)
    dloss = -dprobs[torch.arange(int(num / 2)), dys].log().mean() + (ws.mean() * regularization)
    dloss.detach()

    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * regularization * (abs(loss - dloss)))
    print(loss.item())
    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr

1.2260608673095703
1.234903335571289


RuntimeError: ignored